# Setup

In [62]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import math
import zipfile
import gzip
import shutil
from urllib.request import urlretrieve
import requests
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from keras.layers import StringLookup
from sklearn.preprocessing import LabelEncoder


# Prepare the data

## Download and prepare the DataFrames


In [63]:
URL_MOVIELENS = "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
URL_IMBD_NAMES_BASICS = "https://datasets.imdbws.com/name.basics.tsv.gz"
URL_IMBD_TITLE_BASICS = "https://datasets.imdbws.com/title.basics.tsv.gz"
URL_IMBD_TITLE_RATINGS = "https://datasets.imdbws.com/title.ratings.tsv.gz"
# Local path where the file will be saved
LOCAL_MOVIELENS_PATH = "ml-1m.zip"
LOCAL_IMBD_NAMES_BASICS_PATH = "name.basics.tsv.gz"
LOCAL_IMBD_TITLE_BASICS_PATH = "title.basics.tsv.gz"
LOCAL_IMBD_TITLE_RATINGS_PATH = "title.ratings.tsv.gz"
# Directory where the dataset will be extracted
EXTRACT_DIR = "dataset"

In [64]:
# Function to download the file
def download_file(url, local_filename):
    print(f"Downloading {url} to {local_filename}")
    # Check if the file already exists
    if os.path.exists(local_filename):
        print(f"File {local_filename} already exists")
        return local_filename
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print(f"Downloaded {url} to {local_filename}")
    return local_filename

# Function to unzip the file
def unzip_file(zip_path, extract_to):
    print(f"Unzipping {zip_path} to {extract_to}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Unzipped {zip_path} to {extract_to}")

def gunzip_file(gz_path, extract_to):
    print(f"Gunzipping {gz_path} to {extract_to}")
    with gzip.open(gz_path, 'rb') as f_in:
        with open(extract_to, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Gunzipped {gz_path} to {extract_to}")

In [65]:
# Ensure the dataset directory exists
os.makedirs(EXTRACT_DIR, exist_ok=True)
# Download the file
download_file(URL_MOVIELENS, LOCAL_MOVIELENS_PATH)
download_file(URL_IMBD_NAMES_BASICS, LOCAL_IMBD_NAMES_BASICS_PATH)
download_file(URL_IMBD_TITLE_BASICS, LOCAL_IMBD_TITLE_BASICS_PATH)
download_file(URL_IMBD_TITLE_RATINGS, LOCAL_IMBD_TITLE_RATINGS_PATH)

File ml-1m.zip already exists
File name.basics.tsv.gz already exists
File title.basics.tsv.gz already exists
File title.ratings.tsv.gz already exists


'title.ratings.tsv.gz'

In [66]:
# Extract the files
print("Unzipping file...")
unzip_file(LOCAL_MOVIELENS_PATH, EXTRACT_DIR)

print("Gunzipping files...")
gunzip_file(LOCAL_IMBD_NAMES_BASICS_PATH, os.path.join(EXTRACT_DIR, "name.basics.tsv"))
gunzip_file(LOCAL_IMBD_TITLE_BASICS_PATH, os.path.join(EXTRACT_DIR, "title.basics.tsv"))
gunzip_file(LOCAL_IMBD_TITLE_RATINGS_PATH, os.path.join(EXTRACT_DIR, "title.ratings.tsv"))

print("Extraction complete.")

Unzipping file...
Unzipping ml-1m.zip to dataset
Unzipped ml-1m.zip to dataset
Gunzipping files...
Gunzipping name.basics.tsv.gz to dataset\name.basics.tsv
Gunzipped name.basics.tsv.gz to dataset\name.basics.tsv
Gunzipping title.basics.tsv.gz to dataset\title.basics.tsv
Gunzipped title.basics.tsv.gz to dataset\title.basics.tsv
Gunzipping title.ratings.tsv.gz to dataset\title.ratings.tsv
Gunzipped title.ratings.tsv.gz to dataset\title.ratings.tsv
Extraction complete.


In [67]:
movies = pd.read_csv('dataset/ml-1m/movies.dat', sep='::', header=None, engine='python', names=['movie_id', 'title', 'genres'], encoding='ISO-8859-1')
ratings = pd.read_csv('dataset/ml-1m/ratings.dat', sep='::', header=None, engine='python', names=['user_id', 'movie_id', 'rating', 'unix_timestamp'], encoding='ISO-8859-1')
users = pd.read_csv('dataset/ml-1m/users.dat', sep='::', header=None, engine='python', names=['user_id', 'sex', 'age_group', 'occupation', 'zip_code'], encoding='ISO-8859-1')

In [68]:
title_basics_df = pd.read_csv('dataset/title.basics.tsv', sep='\t', header=0)
title_basics_df = title_basics_df[title_basics_df['titleType'] == 'movie']
name_basics_df = pd.read_csv('dataset/name.basics.tsv', sep='\t', header=0)

C:\Users\marti\AppData\Local\Temp\ipykernel_10024\170924504.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics_df = pd.read_csv('dataset/title.basics.tsv', sep='\t', header=0)


Here, we do some simple data processing to fix the data types of the columns.

In [69]:
users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")

ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

movies['title'] = movies['title'].str.lower()
movies['title'] = movies['title'].str.replace(r"\(.*\)", "", regex=True).str.strip()

title_basics_df['primaryTitle'] = title_basics_df['primaryTitle'].str.lower().str.strip()
title_basics_df.rename(columns={"genres": "genres_title"}, inplace=True)
title_basics_df = title_basics_df.drop_duplicates(subset='primaryTitle')

# Convert primaryName to string
name_basics_df['primaryName'] = name_basics_df['primaryName'].astype(str)

name_basics_df = name_basics_df.assign(knownForTitles=name_basics_df['knownForTitles'].str.split(','))
name_basics_df = name_basics_df.explode('knownForTitles')

# Group by knownForTitles and aggregate primaryName
name_basics_grouped = name_basics_df.groupby('knownForTitles')['primaryName'].agg(lambda x: ','.join(set(x))).reset_index()

# Merge with title_basics_df
title_basics_df = pd.merge(title_basics_df, name_basics_grouped, left_on='tconst', right_on='knownForTitles', how='left')

title_basics_df.drop(columns='knownForTitles', inplace=True)
title_basics_df['primaryName'].fillna('unknown', inplace=True)

C:\Users\marti\AppData\Local\Temp\ipykernel_10024\1417224388.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  title_basics_df['primaryName'].fillna('unknown', inplace=True)


In [70]:
# Merge the dataframes movies and title_basics_df, keep only the the genres_title column and merge on the title column. If the title is not found in title_basics_df, the row is not kept
movies = movies.merge(title_basics_df[['primaryTitle', 'runtimeMinutes','genres_title', 'primaryName']], left_on='title', right_on='primaryTitle', how='left')
# Drop the primaryTitle column
movies.drop(columns=['primaryTitle'], inplace=True)
movies['runtimeMinutes'] = pd.to_numeric(movies['runtimeMinutes'], errors='coerce')
mean_runtime = movies['runtimeMinutes'].mean()
movies['runtimeMinutes'].fillna(mean_runtime, inplace=True)
movies = movies.drop_duplicates()
movies

C:\Users\marti\AppData\Local\Temp\ipykernel_10024\1504217106.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['runtimeMinutes'].fillna(mean_runtime, inplace=True)


,movie_id,title,genres,runtimeMinutes,genres_title,primaryName
0,movie_1,toy story,Animation|Children's|Comedy,81.000000,"Adventure,Animation,Comedy","Spencer Aste,John Morris,Tru'o'ng Gia Tìên,Don..."
1,movie_2,jumanji,Adventure|Children's|Fantasy,104.000000,"Adventure,Comedy,Family","Tracey Boulton,Scott Kroopf,Stan Parks,Melinda..."
2,movie_3,grumpier old men,Comedy|Romance,101.000000,"Comedy,Romance","James Andelin,Wayne A. Evenson,Bix Skahill,Kat..."
3,movie_4,waiting to exhale,Comedy|Drama,124.000000,"Comedy,Drama,Romance","Joann Y. Johnson,Clendolyn Corbin,Jon Gass,Syl..."
4,movie_5,father of the bride part ii,Comedy,106.000000,"Comedy,Family,Romance","Connie McAloney,Robert F. Mann,Thomas Selstad,..."
...,...,...,...,...,...,...
3878,movie_3948,meet the parents,Comedy,75.000000,Comedy,"Jacqueline Cahill,Mary Ruth Clarke,Dick Gallow..."
3879,movie_3949,requiem for a dream,Drama,102.000000,Drama,"Christian Boatsman,Jason Marder,Ralph Fratiann..."
3880,movie_3950,tigerland,Drama,101.000000,"Drama,War","Herman McEachin,Shawn R. McFall,Chris Huvane,F..."
3881,movie_3951,two family house,Drama,108.000000,"Comedy,Drama,Romance","Chelsea Fetzer,John Bunch,Danita Knight,Timoth..."


In [71]:
# Merge the genres column (seperated with | ) with the genres_title column (seperated with ','). Do not keep duplicates
movies['genres'] = movies['genres'].str.split('|')
movies['genres_title'] = movies['genres_title'].str.split(',')
# If the genres_title column is NaN, replace it with an empty list
movies['genres_title'] = movies['genres_title'].apply(lambda x: [] if type(x) == float else x)
movies['genres'] = movies['genres'] + movies['genres_title']
movies['genres'] = movies['genres'].apply(lambda x: list(set(x)))
movies['genres'] = movies['genres'].apply(lambda x: '|'.join(x))
movies.drop(columns=['genres_title'], inplace=True)

In [72]:
movies

,movie_id,title,genres,runtimeMinutes,primaryName
0,movie_1,toy story,Comedy|Children's|Adventure|Animation,81.000000,"Spencer Aste,John Morris,Tru'o'ng Gia Tìên,Don..."
1,movie_2,jumanji,Family|Children's|Comedy|Adventure|Fantasy,104.000000,"Tracey Boulton,Scott Kroopf,Stan Parks,Melinda..."
2,movie_3,grumpier old men,Comedy|Romance,101.000000,"James Andelin,Wayne A. Evenson,Bix Skahill,Kat..."
3,movie_4,waiting to exhale,Comedy|Romance|Drama,124.000000,"Joann Y. Johnson,Clendolyn Corbin,Jon Gass,Syl..."
4,movie_5,father of the bride part ii,Comedy|Romance|Family,106.000000,"Connie McAloney,Robert F. Mann,Thomas Selstad,..."
...,...,...,...,...,...
3878,movie_3948,meet the parents,Comedy,75.000000,"Jacqueline Cahill,Mary Ruth Clarke,Dick Gallow..."
3879,movie_3949,requiem for a dream,Drama,102.000000,"Christian Boatsman,Jason Marder,Ralph Fratiann..."
3880,movie_3950,tigerland,Drama|War,101.000000,"Herman McEachin,Shawn R. McFall,Chris Huvane,F..."
3881,movie_3951,two family house,Drama|Romance|Comedy,108.000000,"Chelsea Fetzer,John Bunch,Danita Knight,Timoth..."


In [73]:
# First, ensure primaryName is a string
movies['primaryName'] = movies['primaryName'].fillna('unknown')
movies['primaryName'] = movies['primaryName'].astype(str)

# Create a LabelEncoder
le = LabelEncoder()

# Fit and transform the primaryName column
movies['primaryName_encoded'] = le.fit_transform(movies['primaryName'])

# Normalize the encoded values to be between 0 and 1
movies['primaryName_encoded'] = movies['primaryName_encoded'] / movies['primaryName_encoded'].max()

# Drop the original primaryName and genres_title columns
movies.drop(columns=['primaryName'], inplace=True)

In [74]:
movies

,movie_id,title,genres,runtimeMinutes,primaryName_encoded
0,movie_1,toy story,Comedy|Children's|Adventure|Animation,81.000000,0.885831
1,movie_2,jumanji,Family|Children's|Comedy|Adventure|Fantasy,104.000000,0.952027
2,movie_3,grumpier old men,Comedy|Romance,101.000000,0.420602
3,movie_4,waiting to exhale,Comedy|Romance|Drama,124.000000,0.473782
4,movie_5,father of the bride part ii,Comedy|Romance|Family,106.000000,0.175158
...,...,...,...,...,...
3878,movie_3948,meet the parents,Comedy,75.000000,0.414652
3879,movie_3949,requiem for a dream,Drama,102.000000,0.156192
3880,movie_3950,tigerland,Drama|War,101.000000,0.381554
3881,movie_3951,two family house,Drama|Romance|Comedy,108.000000,0.150242


Each movie has multiple genres. We split them into separate columns in the movies DataFrame.

In [75]:
movie_features = set()
for actual_genres in movies['genres'].str.split('|'):
    movie_features.update(actual_genres)
movie_features = list(movie_features)

for genre in movie_features:
    movies[genre] = movies["genres"].apply(
        lambda values: int(genre in values.split("|"))
    )

# Add runtimeMinutes and primaryName_encoded to movie_features
movie_features.extend(["runtimeMinutes", "primaryName_encoded"])

In [76]:
movies

,movie_id,title,genres,runtimeMinutes,primaryName_encoded,War,Romance,Horror,Sport,Biography,...,Crime,\N,Drama,Fantasy,Sci-Fi,Action,Mystery,Documentary,Western,History
0,movie_1,toy story,Comedy|Children's|Adventure|Animation,81.000000,0.885831,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,movie_2,jumanji,Family|Children's|Comedy|Adventure|Fantasy,104.000000,0.952027,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,movie_3,grumpier old men,Comedy|Romance,101.000000,0.420602,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,movie_4,waiting to exhale,Comedy|Romance|Drama,124.000000,0.473782,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,movie_5,father of the bride part ii,Comedy|Romance|Family,106.000000,0.175158,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,movie_3948,meet the parents,Comedy,75.000000,0.414652,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,movie_3949,requiem for a dream,Drama,102.000000,0.156192,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3880,movie_3950,tigerland,Drama|War,101.000000,0.381554,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3881,movie_3951,two family house,Drama|Romance|Comedy,108.000000,0.150242,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


### Transform the movie ratings data into sequences
First, let's sort the the ratings data using the unix_timestamp, and then group the movie_id values and the rating values by user_id.

The output DataFrame will have a record for each user_id, with two ordered lists (sorted by rating datetime): the movies they have rated, and their ratings of these movies.

In [77]:
ratings_group = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")

ratings_data = pd.DataFrame(
    data={
        "user_id": list(ratings_group.groups.keys()),
        "movie_ids": list(ratings_group.movie_id.apply(list)),
        "ratings": list(ratings_group.rating.apply(list)),
        "timestamps": list(ratings_group.unix_timestamp.apply(list)),
    }
)

Now, let's split the movie_ids list into a set of sequences of a fixed length. We do the same for the ratings. Set the sequence_length variable to change the length of the input sequence to the model. You can also change the step_size to control the number of sequences to generate for each user.

In [78]:
sequence_length = 4
step_size = 2


def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


ratings_data.movie_ids = ratings_data.movie_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

ratings_data.ratings = ratings_data.ratings.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del ratings_data["timestamps"]

After that, we process the output to have each sequence in a separate records in the DataFrame. In addition, we join the user features with the ratings data.

In [79]:
ratings_data_movies = ratings_data[["user_id", "movie_ids"]].explode(
    "movie_ids", ignore_index=True
)
ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
ratings_data_transformed = pd.concat([ratings_data_movies, ratings_data_rating], axis=1)
ratings_data_transformed = ratings_data_transformed.join(
    users.set_index("user_id"), on="user_id"
)
ratings_data_transformed.movie_ids = ratings_data_transformed.movie_ids.apply(
    lambda x: ",".join(x)
)
ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
    lambda x: ",".join([str(v) for v in x])
)

del ratings_data_transformed["zip_code"]

ratings_data_transformed.rename(
    columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

With sequence_length of 4 and step_size of 2, we end up with 498,623 sequences.

Finally, we split the data into training and testing splits, with 85% and 15% of the instances, respectively, and store them to CSV files.

In [80]:
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85
train_data = ratings_data_transformed[random_selection]
test_data = ratings_data_transformed[~random_selection]

train_data.to_csv(os.path.join(EXTRACT_DIR, "train_data.csv"), index=False, sep="|", header=False)
test_data.to_csv(os.path.join(EXTRACT_DIR, "test_data.csv"), index=False, sep="|", header=False)

## Define metadata

In [81]:
CSV_HEADER = list(ratings_data_transformed.columns)

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "user_id": list(users.user_id.unique()),
    "movie_id": list(movies.movie_id.unique()),
    "sex": list(users.sex.unique()),
    "age_group": list(users.age_group.unique()),
    "occupation": list(users.occupation.unique()),
}

USER_FEATURES = ["sex", "age_group", "occupation"]

MOVIE_FEATURES = ["genres", "runtimeMinutes", "primaryName_encoded"]

## Create tf.data.Dataset for training and evaluation

In [82]:
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        movie_ids_string = features["sequence_movie_ids"]
        sequence_movie_ids = tf.strings.split(movie_ids_string, ",").to_tensor()

        # The last movie id in the sequence is the target movie.
        features["target_movie_id"] = sequence_movie_ids[:, -1]
        features["sequence_movie_ids"] = sequence_movie_ids[:, :-1]

        ratings_string = features["sequence_ratings"]
        sequence_ratings = tf.strings.to_number(
            tf.strings.split(ratings_string, ","), tf.dtypes.float32
        ).to_tensor()

        # The last rating in the sequence is the target for the model to predict.
        target = sequence_ratings[:, -1]
        features["sequence_ratings"] = sequence_ratings[:, :-1]

        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset

## Create model inputs

In [83]:
def create_model_inputs():
    return {
        "user_id": keras.Input(name="user_id", shape=(1,), dtype="string"),
        "sequence_movie_ids": keras.Input(
            name="sequence_movie_ids", shape=(sequence_length - 1,), dtype="string"
        ),
        "target_movie_id": keras.Input(
            name="target_movie_id", shape=(1,), dtype="string"
        ),
        "sequence_ratings": keras.Input(
            name="sequence_ratings", shape=(sequence_length - 1,), dtype=tf.float32
        ),
        "sex": keras.Input(name="sex", shape=(1,), dtype="string"),
        "age_group": keras.Input(name="age_group", shape=(1,), dtype="string"),
        "occupation": keras.Input(name="occupation", shape=(1,), dtype="string"),
    }

## Encode input features
The encode_input_features method works as follows:

1. Each categorical user feature is encoded using layers.Embedding, with embedding dimension equals to the square root of the vocabulary size of the feature. The embeddings of these features are concatenated to form a single input tensor.

2. Each movie in the movie sequence and the target movie is encoded layers.Embedding, where the dimension size is the square root of the number of movies.

3. A multi-hot genres vector for each movie is concatenated with its embedding vector, and processed using a non-linear layers.Dense to output a vector of the same movie embedding dimensions.

4. A positional embedding is added to each movie embedding in the sequence, and then multiplied by its rating from the ratings sequence.

5. The target movie embedding is concatenated to the sequence movie embeddings, producing a tensor with the shape of [batch size, sequence length, embedding size], as expected by the attention layer for the transformer architecture.

6. The method returns a tuple of two elements: encoded_transformer_features and encoded_other_features.

In [84]:
def encode_input_features(
    inputs,
    include_user_id=True,
    include_user_features=True,
    include_movie_features=True,
):
    encoded_transformer_features = []
    encoded_other_features = []

    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ## Encode user features
    for feature_name in other_feature_names:
        # Convert the string input values into integer indices.
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=0)(
            inputs[feature_name]
        )
        # Compute embedding dimensions
        embedding_dims = int(math.sqrt(len(vocabulary)))
        # Create an embedding layer with the specified dimensions.
        embedding_encoder = layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            name=f"{feature_name}_embedding",
        )
        # Convert the index values to embedding representations.
        encoded_other_features.append(embedding_encoder(idx))

    ## Create a single embedding vector for the user features
    if len(encoded_other_features) > 1:
        encoded_other_features = layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ## Create a movie embedding encoder
    movie_vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY["movie_id"]
    movie_embedding_dims = int(math.sqrt(len(movie_vocabulary)))
    # Create a lookup to convert string values to integer indices.
    movie_index_lookup = StringLookup(
        vocabulary=movie_vocabulary,
        mask_token=None,
        num_oov_indices=0,
        name="movie_index_lookup",
    )
    # Create an embedding layer with the specified dimensions.
    movie_embedding_encoder = layers.Embedding(
        input_dim=len(movie_vocabulary),
        output_dim=movie_embedding_dims,
        name=f"movie_embedding",
    )
    # Create a vector lookup for movie features.
    movie_feature_vectors = movies[movie_features].to_numpy()
    movie_features_lookup = layers.Embedding(
        input_dim=movie_feature_vectors.shape[0],
        output_dim=movie_feature_vectors.shape[1],
        embeddings_initializer=keras.initializers.Constant(movie_feature_vectors),
        trainable=False,
        name="features_vector",
    )
    # Create a processing layer for genres.
    movie_embedding_processor = layers.Dense(
        units=movie_embedding_dims,
        activation="relu",
        name="process_movie_embedding_with_genres",
    )

    ## Define a function to encode a given movie id.
    def encode_movie(movie_id):
        # Convert the string input values into integer indices.
        movie_idx = movie_index_lookup(movie_id)
        movie_embedding = movie_embedding_encoder(movie_idx)
        encoded_movie = movie_embedding
        if include_movie_features:
            movie_genres_vector = movie_features_lookup(movie_idx)
            encoded_movie = movie_embedding_processor(
                layers.concatenate([movie_embedding, movie_genres_vector])
            )
        return encoded_movie

    ## Encoding target_movie_id
    target_movie_id = inputs["target_movie_id"]
    encoded_target_movie = encode_movie(target_movie_id)

    ## Encoding sequence movie_ids.
    sequence_movies_ids = inputs["sequence_movie_ids"]
    encoded_sequence_movies = encode_movie(sequence_movies_ids)
    # Create positional embedding.
    position_embedding_encoder = layers.Embedding(
        input_dim=sequence_length,
        output_dim=movie_embedding_dims,
        name="position_embedding",
    )
    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)
    # Retrieve sequence ratings to incorporate them into the encoding of the movie.
    sequence_ratings = inputs["sequence_ratings"]
    sequence_ratings = keras.ops.expand_dims(sequence_ratings, -1)
    # Add the positional encoding to the movie encodings and multiply them by rating.
    encoded_sequence_movies_with_poistion_and_rating = layers.Multiply()(
        [(encoded_sequence_movies + encodded_positions), sequence_ratings]
    )

    # Construct the transformer inputs.
    for i in range(sequence_length - 1):
        feature = encoded_sequence_movies_with_poistion_and_rating[:, i, ...]
        feature = keras.ops.expand_dims(feature, 1)
        encoded_transformer_features.append(feature)
    encoded_transformer_features.append(encoded_target_movie)

    encoded_transformer_features = layers.concatenate(
        encoded_transformer_features, axis=1
    )

    return encoded_transformer_features, encoded_other_features

## Create a BST model

In [85]:
include_user_id = False
include_user_features = True
include_movie_features = True

hidden_units = [256, 128]
dropout_rate = 0.1
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_movie_features
    )

    # Create a multi-headed attention layer.
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block.
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    x1 = layers.Add()([transformer_features, attention_output])
    x1 = layers.LayerNormalization()(x1)
    x2 = layers.LeakyReLU()(x1)
    x2 = layers.Dense(units=x2.shape[-1])(x2)
    x2 = layers.Dropout(dropout_rate)(x2)
    transformer_features = layers.Add()([x1, x2])
    transformer_features = layers.LayerNormalization()(transformer_features)
    features = layers.Flatten()(transformer_features)

    # Included the other features.
    if other_features is not None:
        features = layers.concatenate(
            [features, layers.Reshape([other_features.shape[-1]])(other_features)]
        )

    # Fully-connected layers.
    for num_units in hidden_units:
        features = layers.Dense(num_units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.LeakyReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = create_model()

## Run training and evaluation experiment

In [86]:
# Compile the model.
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv(os.path.join(EXTRACT_DIR, "train_data.csv"), shuffle=True, batch_size=265)

# Fit the model with the training data.
model.fit(train_dataset, epochs=1)

# Read the test data.
test_dataset = get_dataset_from_csv(os.path.join(EXTRACT_DIR, "test_data.csv"), batch_size=265)

# Evaluate the model on the test data.
_, rmse = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(rmse, 3)}")

1600/1600 ━━━━━━━━━━━━━━━━━━━━ 49s 25ms/step - loss: 1.7346 - mean_absolute_error: 1.0337


C:\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


Test MAE: 0.848


In [89]:
for data, target in train_dataset.take(1):
    print(f"Data batch: {data}")
    print(f"Target batch: {target}")
    break

#Print the shape of the batch and the target batch
print(f"Shape of the batch: {data['user_id'].shape}")
print(f"Shape of the target batch: {target.shape}")


Data batch: OrderedDict({'user_id': <tf.Tensor: shape=(265,), dtype=string, numpy=
array([b'user_1086', b'user_1100', b'user_1100', b'user_1069',
       b'user_1088', b'user_1088', b'user_1077', b'user_1050',
       b'user_1069', b'user_1086', b'user_1015', b'user_1015',
       b'user_1001', b'user_1010', b'user_1038', b'user_1067',
       b'user_1010', b'user_1112', b'user_1088', b'user_1100',
       b'user_1044', b'user_1030', b'user_1086', b'user_1040',
       b'user_1105', b'user_1105', b'user_1057', b'user_1113',
       b'user_1045', b'user_1030', b'user_1016', b'user_1015',
       b'user_1114', b'user_1034', b'user_1017', b'user_1065',
       b'user_1036', b'user_10', b'user_1019', b'user_1051', b'user_1100',
       b'user_1096', b'user_1015', b'user_1050', b'user_1117',
       b'user_1100', b'user_1015', b'user_1019', b'user_1093',
       b'user_1023', b'user_1086', b'user_1117', b'user_1010',
       b'user_1001', b'user_1053', b'user_1029', b'user_1074',
       b'user_1069', b'

## We can now make prediction of the ratings

In [87]:
# Use the model to get predictions.
predictions = model.predict(test_dataset)
print(predictions[:1000])
# Print the real target values.
for batch in test_dataset.take(1):
    print(batch[1][:1000])

# Compute the model's MAE
mae = np.mean(np.abs(predictions - batch[1]))
print(f"Mean Absolute Error: {round(mae, 3)}")

283/283 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
[[3.9059536]
 [3.8607876]
 [3.6287692]
 [3.7188241]
 [3.295462 ]
 [3.738754 ]
 [4.0570498]
 [4.028822 ]
 [3.8333988]
 [3.916014 ]
 [4.02771  ]
 [3.9986763]
 [3.917299 ]
 [4.169538 ]
 [3.909094 ]
 [4.158414 ]
 [3.979862 ]
 [3.9790814]
 [4.1220903]
 [4.082652 ]
 [3.6461434]
 [3.480868 ]
 [3.788239 ]
 [3.9285698]
 [4.136835 ]
 [4.044582 ]
 [4.063519 ]
 [3.926677 ]
 [4.0445533]
 [3.263559 ]
 [3.3811424]
 [3.4978669]
 [4.1469774]
 [3.3169234]
 [3.5357444]
 [4.072784 ]
 [3.8539338]
 [4.1740046]
 [3.7741425]
 [3.4695923]
 [3.7610104]
 [3.8211513]
 [3.9995768]
 [2.991216 ]
 [3.8603776]
 [3.415403 ]
 [3.8304906]
 [3.848763 ]
 [3.5194972]
 [4.214438 ]
 [3.5373814]
 [3.8215501]
 [3.3814864]
 [3.7921598]
 [3.3361638]
 [3.322066 ]
 [4.1324005]
 [3.3843677]
 [3.0836675]
 [3.3720732]
 [4.227585 ]
 [4.0935693]
 [3.9549854]
 [3.9802804]
 [4.130663 ]
 [3.9912288]
 [3.6719515]
 [3.705902 ]
 [3.4509466]
 [3.8347912]
 [3.783335 ]
 [3.742699 ]
 [3.0002036]
 [3.65550